## Analizando listado de carros usados en eBay

Trabajeremos un dataset de carros usados de Ebay Kleinanzeigen, una sección de clasificados del website
de eBay Alemania.

El dataset fue originalmente scraped y subido a Kaggle. La versión del dataset que utilizaremos tiene una
muestra de 50000 data points.

A continuación se anexa el contenido de cada una de las columnas del dataset:

* dateCrawled: cuando se rastreó este anuncio por primera vez. Todos los valores de campo se toman a partir
de esta fecha.
* name: nombre del auto.
* seller: si el vendedor es privado o un dealer.
* offerType: tipo de publicación.
* price: el precio de venta del vehículo en la publicación.
* abtest: si el vehículo publicado diposne de pruebas de A/B.
* vehicleType: el tipo de vehículo.
* yearOfRegistration: año en el que auto fue registrado por primera vez.
* gearbox: tipo de transmisión
* powerPS: la potencia del auto en PS.
* model: nombre del modelo del auto.
* kilometer: kilometraje del auto.
* monthOfRegistration: mes en que el auto fue registrado por primera vez.
* fuelType: tipo de combustible que utiliza el auto.
* brand: marca del auto.
* notRepairedDamage: si el auto tiene daños y no han sido reparados aún.
* dateCreated: día en que la publicación del auto fue creada.
* nrOfPictures: número de fotos en la publicación.
* postalCode: código postal de la ubicación donde se encuentra el auto.
* lastSeenOnline: cuando el dueño del anunció revisó el anuncio por última vez.

El propósito de este proyecto es de limpiar la data y analizar la lista de anuncios de carros usados
de eBay Alemania.

In [1]:
import numpy as np
import pandas as pd

In [2]:
auto = pd.read_csv("autos.csv", encoding="Latin-1")
print(auto.info(), "\n")
print(auto.head(),"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

Nuestro dataset tiene 20 columnas, donde la mayoría son de tipo string. Hay algunas columnas que tienen
null values, pero no existen columnas donde un porcentaje mayor del 20% contenga null values. Hay columnas
que tienen fechas que están en formato tipo string.

## Limpieza de las columnas

In [3]:
auto.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Primero haremos par de cambios en las columnas:
* Cambiar el nombre de la columnas de camel case a snake case.
* Modificar un poco el nombre de algunas columnas para que tengas una descripción más precisa.

In [4]:
auto.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
auto.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Ahora comenzaremos la exploración de los datas para ubicar áreas obvias que necesiten limpieza.

In [5]:
auto.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-08 10:40:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Inicialmente podemos observar lo siguiente:
* Hay columnas con la misma información.
    * seller
    * offer_type
* La columna num_photos parece extraña hay que indagar un poco mas al respecto.

In [6]:
print(auto.loc[:, "num_photos"].value_counts(), "\n")
print(auto.loc[:, "seller"].value_counts(), "\n")
print(auto.loc[:, "offer_type"].value_counts(), "\n")


0    50000
Name: num_photos, dtype: int64 

privat        49999
gewerblich        1
Name: seller, dtype: int64 

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64 



Al parecer la columna num_photos tiene un único valor de cero (0) para cada uno de los clasificados, por
esta razón la eliminaremos.

De igual forma vamos a eliminar las columnas seller y offer_type, ya que no ofrecen información relevante y
podemos prescindir de ellas sin ningún tipo de problemas.

In [7]:
auto = auto.drop(["seller", "offer_type", "num_photos"], axis=1)
print(auto.head())

          date_crawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

    price  ab_test vehicle_type  registration_year    gearbox  power_ps  \
0  $5,000  control          bus               2004    manuell       158   
1  $8,500  control    limousine               1997  automatik       286   
2  $8,990     test    limousine               2009    manuell       102   
3  $4,350  control   kleinwagen               2007  automatik        71   
4  $1,350     test        kombi               2003    manuell         0   

    model   odometer  registration_month fuel_type       brand  \
0  andere  150,000km

Podemos ver que en la columna price, que es númerica tiene par de caracteres el "$" y ","
que deben ser eliminados para convertir a la columna a tipo integer (int).

De igual forma vemos que odometer que es una columna númerica tiene un caracter "," y la palabra "km"
que debe de ser eliminados para que la columna pueda convertirse a tipo integer sin problemas.

In [8]:
auto.loc[:,"price"] = auto.loc[:,"price"].str.replace(",", "")
auto.loc[:,"price"] = auto.loc[:,"price"].str.replace("$", "")
auto.loc[:,"price"] = auto.loc[:,"price"].astype(int)

auto.loc[:,"odometer"] = auto.loc[:,"odometer"].str.replace("km", "")
auto.loc[:,"odometer"] = auto.loc[:,"odometer"].str.replace(",", "")
auto.loc[:,"odometer"] = auto.loc[:,"odometer"].astype(int)

auto.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
print(auto.loc[:,["price", "odometer_km"]].head())

   price  odometer_km
0   5000       150000
1   8500       150000
2   8990        70000
3   4350        70000
4   1350       150000


## Explorando "odometer_km" y "price"

In [9]:
print(auto.loc[:,"odometer_km"].value_counts())

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64


Podemos ver que los valores para el kilometraje está preestablecidos en la página de eBay, lo que nos
indica que es probable que los dueños tiene un opción pre-clasificada al momento de realizar la
publicación. De igual forma podemos observar que hay más vehículos de alto kilometraje que de bajo
kilometraje.


In [10]:
print(auto.loc[:,"price"].unique().shape,"\n")
print(auto.loc[:,"price"].describe(), "\n")
print(auto.loc[:,"price"].value_counts().head(20))

(2357,) 

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64 

0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64


Otra vez podemos observar que al existir sólo 2347 datos únicos podemos decir que el kilometraje
se encuentra redondeado, es probable que las personas que publican tengan la tendencia de redondear los
precios. Hay 1421 autos con un precio de 0, que representa solamente un 2% de la muestra total, podemos
considerar quitar esta fila. También podemos observar que el precio máximo es de 100 millones

In [11]:
print(auto.loc[:,"price"].value_counts().sort_index(ascending=False).head(20), "\n")
print(auto.loc[:,"price"].value_counts().sort_index(ascending=True).head(20))


99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64 

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64


Podemos observar que hay una lista de precio de 0 a 35  y mas de 1500 autos publicados entre 0 - 1 , es
de observar que hay muy pocos autos a precios muy altos, alrededor de 14 autos rondan mas del millón. Dado
que eBay es un sitio  de puja es legítimo que existan autos con una oferta inicial de 1, tomando las
premisas anteriores vamos a poner como tope 351000$ como monto máximo en este análisis, ya que el precio
aumenta de manera constante hasta ese monto y luego se va a precios poco realistas (100 millones por
ejemplo)

In [12]:
auto = auto.loc[auto.loc[:,"price"].between(1,351000),:]
print(auto.loc[:, "price"].describe())


count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64


## Explorando la columna date
Podemos ver que hay varias columnas que tiene formato de fecha y son de tipo string como las siguientes:
* date_crawled
* registration_month
* registration_year
* ad_created
* last_seen

In [13]:
print(auto.loc[:,["date_crawled", "ad_created", "last_seen"]].head(5), "\n")
print(auto.loc[:, "date_crawled"].str[0:10].value_counts(normalize=True, dropna=False).sort_index(), "\n")
print(auto.loc[:, "date_crawled"].str[0:10].value_counts(normalize=True, dropna=False).sort_values(), "\n")


          date_crawled           ad_created            last_seen
0  2016-03-26 17:47:46  2016-03-26 00:00:00  2016-04-06 06:45:54
1  2016-04-04 13:38:56  2016-04-04 00:00:00  2016-04-06 14:45:08
2  2016-03-26 18:57:24  2016-03-26 00:00:00  2016-04-06 20:15:37
3  2016-03-12 16:58:10  2016-03-12 00:00:00  2016-03-15 03:16:28
4  2016-04-01 14:38:50  2016-04-01 00:00:00  2016-04-01 14:38:50 

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31

Podemos ver que a partir del de 10 de marzo hasta mediados de abril una cantidad constante de vendedores,
colocaron en venta sus autos, esto lo podemos ver en la distribución que hicimos al normalizar los valores
de value_count.


In [14]:
print(auto.loc[:, "last_seen"].str[0:10].value_counts(normalize=True, dropna=False).sort_index(), "\n")

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64 



El sitio web registra el día en que el dueño de un auto vió la publicación por última vez lo que nos permite
identificar cual el anuncio fue retirado bajo la suposición de que el auto fue vendido.

Los últimos dias (05,06,07 de abril) contienen un cantidad desproporcionada de last seen respecto a los días
anteriores, dado que las que la cantidad de last seen en estos días es 6-10 veces mayor respecto a los días
anteriores, es poco probable que esxistiese un pico masivo de ventas durante esos tres días esto nos muestra
un claro indicativo que tiene que ver con la finalización del anuncio y no la venta de un auto.

In [15]:
#print(auto.loc[:,"ad_created"].str[0:10].unique().shape, "\n")
print(auto.loc[:, "ad_created"].str[0:10].value_counts(normalize=True, dropna=False).sort_index())


2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64


Hay una amplia variedad de días de creación de anuncios para la venta de carros usados, son un poco mas de dos
meses aproximadamente, la mayoría de los anuncios tienes una antiguedad de creación de 1-2 meses, hay algunos
que son muy viejos con el anuncio mas viejo que tiene una antiguedad de 9 meses aproximadamente.

In [16]:
auto.loc[:,"registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

El día cuando el auto fue registrado por primera vez indica la edad de éste, observando la información tenemos
datos que lucen extraños, el valor mínimo es de 1000, mucho antes en que los autos fueron inventados,
y tenemos como valor máximo 9999 mucho mas adelante en el futuro.

 ## Tratamiendo de registration year con datos incorrectos
Todo auto con una fecha de registro por encima del año 2016 es inpreciso, averiguar exactamente cuando fue
creado el primer auto es un poco difícil pero fue durante las primeras décadas de 1900, la opción mas viable
es eliminar estos anuncios cono estas fechas, vamos a determinar cual es el porcentaje de data errónea tenemos
para determinar si eliminamos o no los registros.

In [17]:
print((~auto.loc[:,"registration_year"].between(1900, 2016)).sum()/auto.shape[0])

0.038793369710697


Como representa menos del 4%, vamos a remover todos aquellos auntos que no se encuentren dentro del rango
1900 a 2016.


In [18]:
auto = auto.loc[auto.loc[:,"registration_year"].between(1900,2016),:]
print(auto.loc[:,"registration_year"].value_counts(normalize=True).sort_index(), "\n")
print(auto.loc[:,"registration_year"].value_counts(normalize=True).sort_index(ascending=False).head(20))
print(auto.loc[:,"registration_year"].value_counts(normalize=True).sort_index(ascending=False).head(20).sum())

1910    0.000107
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
          ...   
2012    0.028063
2013    0.017202
2014    0.014203
2015    0.008397
2016    0.026135
Name: registration_year, Length: 78, dtype: float64 

2016    0.026135
2015    0.008397
2014    0.014203
2013    0.017202
2012    0.028063
2011    0.034768
2010    0.034040
2009    0.044665
2008    0.047450
2007    0.048778
2006    0.057197
2005    0.062895
2004    0.057904
2003    0.057818
2002    0.053255
2001    0.056468
2000    0.067608
1999    0.062060
1998    0.050620
1997    0.041794
Name: registration_year, dtype: float64
0.8713180951564876


Se puede inferir que la mayoría de los autos (aproximadamente 87%) fueron registrados en los ultimos 20 años.

## Explorando precios de autos por marca


In [19]:
print(auto.loc[:,"brand"].value_counts(normalize=True))

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

Las ensambladoras alemanas conforman el top 5 en marcas de autos, estas son primero volkswagen, bmw, opel,
mercedes benz, y en la quinta casilla se encuentra audi. Estas 5 marcas representan aproximadamente el 50%,
de los autos publicados. Podemos ver que volkswagen por lejos es la marca de autos mas popular en el site de
eBay con eproximadamente el doble de autos que bmw y opel combinados. Tenemos marcas que tienen un porcentaje
insignificantes de autos, por lo tanto limitaremos nuestr análisis a todoas aquellas marcas que tengan un 5%
porciento o más del total de autos en el listado.

In [20]:
brand_counts = auto.loc[:,"brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts >0.05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [21]:
brand_price_average = {}
for brand in common_brands:
    brand_only = auto.loc[auto.loc[:,"brand"] == brand, :]
    price_average = brand_only.loc[:,"price"].mean()
    brand_price_average[brand] = int(price_average)
print(brand_price_average)

{'volkswagen': 5402, 'bmw': 8332, 'opel': 2975, 'mercedes_benz': 8628, 'audi': 9336, 'ford': 3749}


Entre las marcas populares de carros usados encontradas tenemos los siguientes hallazgos en precio:
* Audi, Mercedes Benz y Bmw son los autos mas costosos.
* Ford y Opel son los las económicos
* Volkswagen está en la mitad del rango de precios, esto puede explicar su popularidad, ya que pueden ofrecer
las mejores características de ambos segmentos (calidad - precio).


## Explorando el Kilometraje (Odometer)



In [22]:
bmp = pd.Series(brand_price_average).sort_values(ascending=False)
pd.DataFrame(bmp, columns=["mean_price"])


,mean_price
audi,9336
mercedes_benz,8628
bmw,8332
volkswagen,5402
ford,3749
opel,2975


In [23]:
brand_mileage_average = {}
for brand in common_brands:
    brand_only = auto.loc[auto.loc[:,"brand"] == brand, :]
    mile_average = brand_only.loc[:,"odometer_km"].mean()
    brand_mileage_average[brand] = int(mile_average)
print(brand_mileage_average)

{'volkswagen': 128707, 'bmw': 132572, 'opel': 129310, 'mercedes_benz': 130788, 'audi': 129157, 'ford': 124266}


In [24]:
bma = pd.Series(brand_mileage_average).sort_values(ascending=False)
bma_df = pd.DataFrame(bma, columns=["mean_mileage"])
print(bma_df)


               mean_mileage
bmw                  132572
mercedes_benz        130788
opel                 129310
audi                 129157
volkswagen           128707
ford                 124266


In [25]:
final_table = bma_df
final_table["mean_price"] = bmp
print(final_table)

               mean_mileage  mean_price
bmw                  132572        8332
mercedes_benz        130788        8628
opel                 129310        2975
audi                 129157        9336
volkswagen           128707        5402
ford                 124266        3749


El rango de kilometraje de los automóviles no varía tanto como los precios por marca, sino que todos caen
dentro del 10% para las mejores marcas. Existe una ligera tendencia a que los vehículos más costosoa tengan
un mayor kilometraje, y los vehículos menos costosos tengan un menor kilometraje.